In [26]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd 
import seaborn as sns


from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
import os

In [27]:
train = pd.read_csv('train_prepared.csv', index_col=[0], parse_dates=[0])
#test = pd.read_csv('test_prepared.csv', index_col=[0], parse_dates=[0])

train.head()

,Passengers,year,month,dayofmonth,quarter,dayofyear,weekofyear,dayofweek,hour,t1,t2,t3,exp_0_01,exp_0_03,exp_0_08
Month,,,,,,,,,,,,,,,
1949-01-01,112,1949,1,1,1,1,53,5,0,0.0,0.0,0.0,112.000000,112.000000,112.000000
1949-02-01,118,1949,2,1,1,32,5,1,0,112.0,0.0,0.0,112.060000,112.180000,112.480000
1949-03-01,132,1949,3,1,1,60,9,1,0,118.0,112.0,0.0,112.259400,112.774600,114.041600
1949-04-01,129,1949,4,1,2,91,13,4,0,132.0,118.0,112.0,112.426806,113.261362,115.238272
1949-05-01,121,1949,5,1,2,121,17,6,0,129.0,132.0,118.0,112.512538,113.493521,115.699210


Сделаем кроссвалидацию и сохраним оценки

In [28]:
lr = LinearRegression()
sc = StandardScaler()
X = train.drop('Passengers', 1).values
y = train['Passengers'].values
scaler = sc.fit(X)
X = sc.transform(X)

In [29]:
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(10)

CV_scores_lr = []
for train_index, test_index in tscv.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    preds = lr.predict(X_test)
    CV_scores_lr.append(r2_score(y_test, preds))

TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] TEST: [20 21 22 23 24 25 26 27 28 29 30]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30] TEST: [31 32 33 34 35 36 37 38 39 40 41]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41] TEST: [42 43 44 45 46 47 48 49 50 51 52]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52] TEST: [53 54 55 56 57 58 59 60 61 62 63]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63] TEST: [64 65 66 67 68 69 70 71 72 73 74]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 

In [30]:
CV_scores_lr

[0.8948775485508446,
 -0.8694451624714359,
 -13.426257654675258,
 -0.2670670395446195,
 0.29345276476752236,
 -0.5200864889948722,
 0.7166268546539447,
 0.6130361636018183,
 0.731110006246132,
 0.8518915088998802]

Сохраним обученную модель

In [31]:
import dill

In [32]:
lr = LinearRegression()
lr.fit(X, y)
with open("models/LR.model", "wb") as f:
    dill.dump(lr, f)

### Ridge regression

In [33]:
tscv = TimeSeriesSplit(10)

CV_scores_lr_l2 = []
for train_index, test_index in tscv.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lr = Ridge()
    lr.fit(X_train, y_train)
    preds = lr.predict(X_test)
    CV_scores_lr_l2.append(r2_score(y_test, preds))

TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] TEST: [20 21 22 23 24 25 26 27 28 29 30]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30] TEST: [31 32 33 34 35 36 37 38 39 40 41]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41] TEST: [42 43 44 45 46 47 48 49 50 51 52]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52] TEST: [53 54 55 56 57 58 59 60 61 62 63]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63] TEST: [64 65 66 67 68 69 70 71 72 73 74]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 

In [34]:
lr = Ridge()
lr.fit(X, y)
with open("models/LR_l2.model", "wb") as f:
    dill.dump(lr, f)

### Lasso regression

In [35]:
tscv = TimeSeriesSplit(10)

CV_scores_lr_l1 = []
for train_index, test_index in tscv.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lr = Lasso()
    lr.fit(X_train, y_train)
    preds = lr.predict(X_test)
    CV_scores_lr_l1.append(r2_score(y_test, preds))
    
lr = Lasso()
lr.fit(X, y)
with open("models/LR_l1.model", "wb") as f:
    dill.dump(lr, f)

TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] TEST: [20 21 22 23 24 25 26 27 28 29 30]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30] TEST: [31 32 33 34 35 36 37 38 39 40 41]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41] TEST: [42 43 44 45 46 47 48 49 50 51 52]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52] TEST: [53 54 55 56 57 58 59 60 61 62 63]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63] TEST: [64 65 66 67 68 69 70 71 72 73 74]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 

In [36]:
pd.DataFrame({'LR': CV_scores_lr, 
              'LR_l2': CV_scores_lr_l1,
              'LR_l1': CV_scores_lr_l2}).to_csv("./models/CV_LR.csv")

In [37]:
pd.DataFrame({'LR': CV_scores_lr, 
              'LR_l2': CV_scores_lr_l1,
              'LR_l1': CV_scores_lr_l2})

,LR,LR_l2,LR_l1
0,0.894878,-0.436583,0.022265
1,-0.869445,-0.107024,0.371654
2,-13.426258,-0.273164,0.066631
3,-0.267067,0.216795,0.141724
4,0.293453,0.238687,0.358970
5,-0.520086,0.138947,0.313083
6,0.716627,0.314448,0.540140
7,0.613036,0.495240,0.612003
8,0.731110,0.509529,0.593427
9,0.851892,0.653805,0.694711


## ПЕРЕРЫВ до 21:00